In [1]:
# Import necessary libraries for data manipulation and machine learning
import pandas as pd
import numpy as np
import torch

In [31]:
# Load the BindDB dataset from a PyTorch file
bind_db = torch.load('D:\Drugllm\Drugtargetdata\BindDB.pt', weights_only=False)

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Cindy\AppData\Local\Temp\ipykernel_17808\2060436183.py:2: SyntaxWarning: invalid escape sequence '\D'
  bind_db = torch.load('D:\Drugllm\Drugtargetdata\BindDB.pt', weights_only=False)


In [3]:
# Load the Davis dataset from a PyTorch file
davis_db = torch.load('D:\Drugllm\Drugtargetdata\Davis.pt', weights_only=False)

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Cindy\AppData\Local\Temp\ipykernel_17808\503051307.py:2: SyntaxWarning: invalid escape sequence '\D'
  davis_db = torch.load('D:\Drugllm\Drugtargetdata\Davis.pt', weights_only=False)


In [ ]:
# Load the Kiba dataset from a PyTorch file
kiba_db = torch.load('D:\Drugllm\Drugtargetdata\Kiba.pt', weights_only=False)

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Cindy\AppData\Local\Temp\ipykernel_6200\1919925107.py:1: SyntaxWarning: invalid escape sequence '\D'
  kiba_db = torch.load('D:\Drugllm\Kiba.pt',weights_only=False)


In [4]:
# Import additional libraries for machine learning and evaluation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib
from xgboost import XGBRegressor

In [7]:
# Initialize a DataFrame to store evaluation metrics
metrics_df = pd.DataFrame(columns=['Model', 'Dataset', 'RMSE', 'MAE', 'MSE', 'R2'])

def calculate_metrics(y_true, y_pred):
    """Calculate regression metrics: RMSE, MAE, MSE, and R2."""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, mse, r2


def evaluate_model(model_name, model, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=False, predictions_dir='D:/Drugllm/predictions',datasetname="davis"):
    """Train and evaluate a model, save metrics to the DataFrame, and optionally save predictions to CSV.

    Parameters
    ----------
    model_name : str
        Human-readable name of the model (used in metrics and filenames).
    model : object
        Fitted model object with a .predict method.
    X_train, X_val, X_test : array-like
        Feature matrices for each split.
    y_train, y_val, y_test : array-like or pandas.Series
        True target values for each split.
    save_predictions : bool, optional
        If True, save per-split prediction CSV files to `predictions_dir`.
    predictions_dir : str, optional
        Directory where prediction CSVs will be saved.

    Returns
    -------
    pandas.DataFrame
        Updated metrics DataFrame (appended to global `metrics_df`).
    """
    global metrics_df
    import os
    os.makedirs(predictions_dir, exist_ok=True)

    # Train predictions
    train_pred = model.predict(X_train)
    train_metrics = calculate_metrics(y_train, train_pred)

    # Validation predictions
    val_pred = model.predict(X_val)
    val_metrics = calculate_metrics(y_val, val_pred)

    # Test predictions
    test_pred = model.predict(X_test)
    test_metrics = calculate_metrics(y_test, test_pred)

    # Optionally save predictions to CSV files (true vs predicted)
    if save_predictions:
        try:
            # Ensure arrays are 1-D
            y_train_arr = np.array(y_train).flatten()
            y_val_arr = np.array(y_val).flatten()
            y_test_arr = np.array(y_test).flatten()
            train_pred_arr = np.array(train_pred).flatten()
            val_pred_arr = np.array(val_pred).flatten()
            test_pred_arr = np.array(test_pred).flatten()

            # Prepare DataFrames for each split
            train_df_pred = pd.DataFrame({'y_true': y_train_arr, 'y_pred': train_pred_arr})
            val_df_pred = pd.DataFrame({'y_true': y_val_arr, 'y_pred': val_pred_arr})
            test_df_pred = pd.DataFrame({'y_true': y_test_arr, 'y_pred': test_pred_arr})

            # File paths
            safe_name = model_name.replace(' ', '_')
            train_fp = os.path.join(predictions_dir, f'{safe_name}_train_predictions_{datasetname}.csv')
            val_fp = os.path.join(predictions_dir, f'{safe_name}_val_predictions_{datasetname}.csv')
            test_fp = os.path.join(predictions_dir, f'{safe_name}_test_predictions_{datasetname}.csv')

            # Save to CSV
            train_df_pred.to_csv(train_fp, index=False)
            val_df_pred.to_csv(val_fp, index=False)
            test_df_pred.to_csv(test_fp, index=False)
        except Exception as e:
            print(f'Warning: could not save predictions for {model_name}: {e}')

    # Append metrics to the DataFrame
    datasets = ['Training', 'Validation', 'Test']
    metrics = [train_metrics, val_metrics, test_metrics]
    rows = []
    for dataset, metric in zip(datasets, metrics):
        rmse, mae, mse, r2 = metric
        rows.append({
            'Model': model_name,
            'Dataset': dataset,
            'RMSE': rmse,
            'MAE': mae,
            'MSE': mse,
            'R2': r2
        })

    metrics_df = pd.concat([metrics_df, pd.DataFrame(rows)], ignore_index=True)
    return metrics_df


In [13]:
# Set a random seed for reproducibility
SEED = 42

# Create a copy of the Davis dataset for training and testing
dti_data = davis_db.copy().reset_index(drop=True)

# Split the dataset into training, validation, and test sets
train_df, temp_df = train_test_split(dti_data, test_size=0.3, random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED)

# Prepare feature matrices and target variables
X_train = np.hstack((np.vstack(train_df['Drug_Features']), np.vstack(train_df['Target_Features'])))
X_val = np.hstack((np.vstack(val_df['Drug_Features']), np.vstack(val_df['Target_Features'])))
X_test = np.hstack((np.vstack(test_df['Drug_Features']), np.vstack(test_df['Target_Features'])))
y_train, y_val, y_test = train_df['Affinity'], val_df['Affinity'], test_df['Affinity']

# Display the shapes of the feature matrices
print(X_train.shape, X_val.shape, X_test.shape)

(18040, 1088) (3866, 1088) (3866, 1088)


In [10]:
dti_data.shape

(25772, 8)

In [13]:
dti_data.loc[1]['Drug_Features'].shape

(768,)

In [14]:
dti_data.loc[1]['Target_Features'].shape

(320,)

In [6]:
dti_data.head()

,Drug_ID,Drug,Target_ID,Target,Y,Affinity,Drug_Features,Target_Features
0,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,AAK1,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,43.0,7.366532,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.009276379, -0.14671455, 0.065141745, 0.037..."
1,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ABL1p,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,10000.0,5.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.052989967, -0.19866328, 0.01483496, 0.0618..."
2,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ABL2,MVLGTVLLPPNSYGRDQDTSLCCLCTEASESALPDLTDHFASCVED...,10000.0,5.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.028185444, -0.17497379, 0.061383072, 0.063..."
3,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ACVR1,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,10000.0,5.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[0.060061518, -0.12135677, 0.12382316, 0.06401..."
4,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ACVR2A,MGAAAKLAFAVFLISCSSGAILGRSETQECLFFNANWEKDRTNQTG...,10000.0,5.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[0.10380349, -0.11144593, 0.11350971, 0.110970..."


In [14]:
# Scale the feature matrices for models that require normalized input
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [15]:
# Train and evaluate a Random Forest model
print('Random Forest')
rf_model = RandomForestRegressor(n_estimators=100, random_state=SEED)
rf_model.fit(X_train_scaled, y_train)
evaluate_model("Random Forest", rf_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, save_predictions=True, predictions_dir='./Prediction_Results',datasetname="davis")

Random Forest


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161


In [16]:
# Train and evaluate an SVR model
print('SVR')
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train_scaled, y_train)
evaluate_model('SVR', svr_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test)

SVR


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481


In [17]:
# Train and evaluate a Gradient Boosting Machine (GBM) model
print('GBM')
gbm_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=SEED)
gbm_model.fit(X_train, y_train)
evaluate_model('GBM', gbm_model, X_train, y_train, X_val, y_val, X_test, y_test)

GBM


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481
9,GBM,Training,0.610596,0.387668,0.372827,0.460126


In [18]:
# Train and evaluate a Linear Regression model
print('Linear Regression')
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_train_scaled, y_train)
evaluate_model('Linear Regression', lin_reg_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test)

Linear Regression


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481
9,GBM,Training,0.610596,0.387668,0.372827,0.460126


In [19]:
# Train and evaluate a Multi-Layer Perceptron (MLP) model
print('MLP')
mlp_model = MLPRegressor(hidden_layer_sizes=(512, 256), activation='relu', max_iter=200, random_state=SEED)
mlp_model.fit(X_train_scaled, y_train)
evaluate_model('MLP', mlp_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test)

MLP


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481
9,GBM,Training,0.610596,0.387668,0.372827,0.460126


In [20]:
# Train and evaluate an XGBoost model
print('XGBoost')
xgb_model = XGBRegressor(random_state=SEED, eval_metric='rmse')
xgb_model.fit(X_train, y_train)
evaluate_model('XGBoost', xgb_model, X_train, y_train, X_val, y_val, X_test, y_test)

XGBoost


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481
9,GBM,Training,0.610596,0.387668,0.372827,0.460126


In [21]:
# Save the evaluation metrics for the Davis dataset to a CSV file
metrics_df.to_csv('D:\\Github\\llm-drug-agent\\Prediction_Results\\davis_metrics.csv', index=False)

In [22]:
SEED = 42
dti_data = bind_db.copy()
# Split into train, validation, and test sets
train_df, temp_df = train_test_split(dti_data, test_size=0.3, random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED)

X_train = np.hstack((np.vstack(train_df['Drug_Features']), np.vstack(train_df['Target_Features'])))
X_val = np.hstack((np.vstack(val_df['Drug_Features']), np.vstack(val_df['Target_Features'])))
X_test = np.hstack((np.vstack(test_df['Drug_Features']), np.vstack(test_df['Target_Features'])))
y_train, y_val, y_test = train_df['Affinity'], val_df['Affinity'], test_df['Affinity']
print(X_train.shape, X_val.shape, X_test.shape)

(29565, 1088) (6335, 1088) (6336, 1088)


In [23]:
# Scale features for certain models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [25]:
print('Running models...')

print('Random Forest')
# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=SEED)
rf_model.fit(X_train, y_train)
evaluate_model('Random Forest', rf_model, X_train, y_train, X_val, y_val, X_test, y_test)

Running models...
Random Forest


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.221362,0.132025,0.049001,0.929044
1,Random Forest,Validation,0.575139,0.346981,0.330785,0.504049
2,Random Forest,Test,0.610381,0.368079,0.372565,0.478595
3,SVR,Training,0.576812,0.295331,0.332712,0.518214
4,SVR,Validation,0.616045,0.338454,0.379512,0.430992
5,SVR,Test,0.666415,0.363452,0.444108,0.378471
6,GBM,Training,0.610596,0.387668,0.372827,0.460126
7,GBM,Validation,0.625936,0.395796,0.391796,0.412573
8,GBM,Test,0.669750,0.418549,0.448565,0.372235
9,Linear Regression,Training,0.659365,0.431437,0.434762,0.370440


In [24]:
# Train and evaluate a Random Forest model
print('Random Forest')
rf_model = RandomForestRegressor(n_estimators=100, random_state=SEED)
rf_model.fit(X_train_scaled, y_train)
evaluate_model("Random Forest", rf_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, save_predictions=True, predictions_dir='./Prediction_Results',datasetname="binddb")

Random Forest


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481
9,GBM,Training,0.610596,0.387668,0.372827,0.460126


In [25]:
print('SVR')
# SVR
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train_scaled, y_train)
evaluate_model('SVR', svr_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test)

SVR


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481
9,GBM,Training,0.610596,0.387668,0.372827,0.460126


In [26]:
print('GBM')
# GBM
gbm_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=SEED)
gbm_model.fit(X_train, y_train)
evaluate_model('GBM', gbm_model, X_train, y_train, X_val, y_val, X_test, y_test)

GBM


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481
9,GBM,Training,0.610596,0.387668,0.372827,0.460126


In [27]:
print('Linear Regression')
# Linear Regression
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_train_scaled, y_train)
evaluate_model('Linear Regression', lin_reg_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test)

Linear Regression


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481
9,GBM,Training,0.610596,0.387668,0.372827,0.460126


In [28]:
print('MLP')
# MLP
mlp_model = MLPRegressor(hidden_layer_sizes=(512, 256), activation='relu', max_iter=200, random_state=SEED)
mlp_model.fit(X_train_scaled, y_train)
evaluate_model('MLP', mlp_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test)

MLP


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481
9,GBM,Training,0.610596,0.387668,0.372827,0.460126


In [29]:
print('XGBoost')
# XGBoost
xgb_model = XGBRegressor(random_state=SEED, eval_metric='rmse')
xgb_model.fit(X_train, y_train)
evaluate_model('XGBoost', xgb_model, X_train, y_train, X_val, y_val, X_test, y_test)

XGBoost


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.298590,0.226393,0.089156,0.911556
1,Random Forest,Validation,0.361148,0.320461,0.130428,0.553624
2,Random Forest,Test,0.958609,0.732419,0.918931,0.474695
3,Random Forest,Training,0.220814,0.131651,0.048759,0.929394
4,Random Forest,Validation,0.574991,0.347334,0.330615,0.504304
5,Random Forest,Test,0.608878,0.367512,0.370732,0.481161
6,SVR,Training,0.576809,0.295333,0.332708,0.518220
7,SVR,Validation,0.616044,0.338453,0.379511,0.430993
8,SVR,Test,0.666409,0.363454,0.444101,0.378481
9,GBM,Training,0.610596,0.387668,0.372827,0.460126


In [30]:
# Save the evaluation metrics for the BindDB dataset to a CSV file
metrics_df.to_csv('D:\\Github\\llm-drug-agent\\Prediction_Results\\binddb_metrics.csv', index=False)

In [37]:
SEED = 42
dti_data = kiba_db.copy()
# Split into train, validation, and test sets
train_df, temp_df = train_test_split(dti_data, test_size=0.3, random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED)

X_train = np.hstack((np.vstack(train_df['Drug_Features']), np.vstack(train_df['Target_Features'])))
X_val = np.hstack((np.vstack(val_df['Drug_Features']), np.vstack(val_df['Target_Features'])))
X_test = np.hstack((np.vstack(test_df['Drug_Features']), np.vstack(test_df['Target_Features'])))
y_train, y_val, y_test = train_df['Affinity'], val_df['Affinity'], test_df['Affinity']
print(X_train.shape, X_val.shape, X_test.shape)

(82359, 1088) (17649, 1088) (17649, 1088)


In [38]:
# Scale features for certain models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [39]:
print('Running models...')

print('Random Forest')
# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=SEED)
rf_model.fit(X_train, y_train)
evaluate_model('Random Forest', rf_model, X_train, y_train, X_val, y_val, X_test, y_test)

Running models...
Random Forest


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.221362,0.132025,0.049001,0.929044
1,Random Forest,Validation,0.575139,0.346981,0.330785,0.504049
2,Random Forest,Test,0.610381,0.368079,0.372565,0.478595
3,SVR,Training,0.576812,0.295331,0.332712,0.518214
4,SVR,Validation,0.616045,0.338454,0.379512,0.430992
5,SVR,Test,0.666415,0.363452,0.444108,0.378471
6,GBM,Training,0.610596,0.387668,0.372827,0.460126
7,GBM,Validation,0.625936,0.395796,0.391796,0.412573
8,GBM,Test,0.669750,0.418549,0.448565,0.372235
9,Linear Regression,Training,0.659365,0.431437,0.434762,0.370440


In [40]:
print('SVR')
# SVR
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train_scaled, y_train)
evaluate_model('SVR', svr_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test)

SVR


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.221362,0.132025,0.049001,0.929044
1,Random Forest,Validation,0.575139,0.346981,0.330785,0.504049
2,Random Forest,Test,0.610381,0.368079,0.372565,0.478595
3,SVR,Training,0.576812,0.295331,0.332712,0.518214
4,SVR,Validation,0.616045,0.338454,0.379512,0.430992
5,SVR,Test,0.666415,0.363452,0.444108,0.378471
6,GBM,Training,0.610596,0.387668,0.372827,0.460126
7,GBM,Validation,0.625936,0.395796,0.391796,0.412573
8,GBM,Test,0.669750,0.418549,0.448565,0.372235
9,Linear Regression,Training,0.659365,0.431437,0.434762,0.370440


In [41]:
print('GBM')
# GBM
gbm_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=SEED)
gbm_model.fit(X_train, y_train)
evaluate_model('GBM', gbm_model, X_train, y_train, X_val, y_val, X_test, y_test)

GBM


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.221362,0.132025,0.049001,0.929044
1,Random Forest,Validation,0.575139,0.346981,0.330785,0.504049
2,Random Forest,Test,0.610381,0.368079,0.372565,0.478595
3,SVR,Training,0.576812,0.295331,0.332712,0.518214
4,SVR,Validation,0.616045,0.338454,0.379512,0.430992
5,SVR,Test,0.666415,0.363452,0.444108,0.378471
6,GBM,Training,0.610596,0.387668,0.372827,0.460126
7,GBM,Validation,0.625936,0.395796,0.391796,0.412573
8,GBM,Test,0.669750,0.418549,0.448565,0.372235
9,Linear Regression,Training,0.659365,0.431437,0.434762,0.370440


In [42]:
print('Linear Regression')
# Linear Regression
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_train_scaled, y_train)
evaluate_model('Linear Regression', lin_reg_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test)

Linear Regression


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.221362,0.132025,0.049001,0.929044
1,Random Forest,Validation,0.575139,0.346981,0.330785,0.504049
2,Random Forest,Test,0.610381,0.368079,0.372565,0.478595
3,SVR,Training,0.576812,0.295331,0.332712,0.518214
4,SVR,Validation,0.616045,0.338454,0.379512,0.430992
5,SVR,Test,0.666415,0.363452,0.444108,0.378471
6,GBM,Training,0.610596,0.387668,0.372827,0.460126
7,GBM,Validation,0.625936,0.395796,0.391796,0.412573
8,GBM,Test,0.669750,0.418549,0.448565,0.372235
9,Linear Regression,Training,0.659365,0.431437,0.434762,0.370440


In [43]:
print('MLP')
# MLP
mlp_model = MLPRegressor(hidden_layer_sizes=(512, 256), activation='relu', max_iter=200, random_state=SEED)
mlp_model.fit(X_train_scaled, y_train)
evaluate_model('MLP', mlp_model, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test)

MLP


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.221362,0.132025,0.049001,0.929044
1,Random Forest,Validation,0.575139,0.346981,0.330785,0.504049
2,Random Forest,Test,0.610381,0.368079,0.372565,0.478595
3,SVR,Training,0.576812,0.295331,0.332712,0.518214
4,SVR,Validation,0.616045,0.338454,0.379512,0.430992
5,SVR,Test,0.666415,0.363452,0.444108,0.378471
6,GBM,Training,0.610596,0.387668,0.372827,0.460126
7,GBM,Validation,0.625936,0.395796,0.391796,0.412573
8,GBM,Test,0.669750,0.418549,0.448565,0.372235
9,Linear Regression,Training,0.659365,0.431437,0.434762,0.370440


In [44]:
print('XGBoost')
# XGBoost
xgb_model = XGBRegressor(random_state=SEED, eval_metric='rmse')
xgb_model.fit(X_train, y_train)
evaluate_model('XGBoost', xgb_model, X_train, y_train, X_val, y_val, X_test, y_test)

XGBoost


,Model,Dataset,RMSE,MAE,MSE,R2
0,Random Forest,Training,0.221362,0.132025,0.049001,0.929044
1,Random Forest,Validation,0.575139,0.346981,0.330785,0.504049
2,Random Forest,Test,0.610381,0.368079,0.372565,0.478595
3,SVR,Training,0.576812,0.295331,0.332712,0.518214
4,SVR,Validation,0.616045,0.338454,0.379512,0.430992
5,SVR,Test,0.666415,0.363452,0.444108,0.378471
6,GBM,Training,0.610596,0.387668,0.372827,0.460126
7,GBM,Validation,0.625936,0.395796,0.391796,0.412573
8,GBM,Test,0.669750,0.418549,0.448565,0.372235
9,Linear Regression,Training,0.659365,0.431437,0.434762,0.370440


# Exporting Metrics to CSV
The following code saves the metrics DataFrame to a CSV file for further analysis or sharing.

In [ ]:
# Save the metrics DataFrame to a CSV file
metrics_df.to_csv('D:/Drugllm/kiba_metrics.csv', index=False)

In [ ]:
def extract_target_features(sequence):
    """Extracts features from a protein sequence."""
    if sequence is None:
        return None
    # Example feature extraction: Sequence length and count of Alanine (A)
    seq_length = len(sequence)
    alanine_count = sequence.count('A')
    return [seq_length, alanine_count]

# Apply the function to the 'Target' column of bind_db to create a new column 'Extracted_Target_Features'
bind_db['Extracted_Target_Features'] = bind_db['Target'].apply(extract_target_features)

# Display the updated DataFrame with the new column
display(bind_db.head())

In [ ]:
# Drop rows where feature extraction failed for either drug or target
bind_db_cleaned = bind_db.dropna(subset=['Extracted_Drug_Features', 'Extracted_Target_Features']).copy()

# Combine drug and target features into a single feature matrix
# Ensure that the features are in a format suitable for machine learning models (e.g., numpy arrays)
X_drug = np.array(bind_db_cleaned['Extracted_Drug_Features'].tolist())
X_target = np.array(bind_db_cleaned['Extracted_Target_Features'].tolist())

X = np.hstack((X_drug, X_target))

# Create the target variable
y = bind_db_cleaned['Affinity'].values

print("Shape of features (X):", X.shape)
print("Shape of target (y):", y.shape)

Shape of features (X): (42236, 5)
Shape of target (y): (42236,)


In [ ]:
def extract_drug_features(drug):
    """Extracts features from a drug representation (e.g., SMILES string)."""
    if drug is None:
        return None
    # Example feature extraction: Molecular weight, LogP, and number of rings (simplified)
    # These are just placeholders; a real implementation would use a cheminformatics library like RDKit
    mol_weight = len(drug) * 10  # Placeholder
    logp = len(drug) * 0.1 # Placeholder
    num_rings = drug.count('=') # Placeholder
    return [mol_weight, logp, num_rings]

# Apply the function to the 'Drug' column of davis_db to create a new column 'Extracted_Drug_Features'
davis_db['Extracted_Drug_Features'] = davis_db['Drug'].apply(extract_drug_features)

# Display the updated DataFrame with the new column
display(davis_db.head())

,Drug_ID,Drug,Target_ID,Target,Y,Affinity,Drug_Features,Target_Features,Extracted_Drug_Features
0,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,AAK1,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,43.0,-1.633468,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.009276379, -0.14671455, 0.065141745, 0.037...","[450, 4.5, 0]"
1,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ABL1p,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,10000.0,-4.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.052989967, -0.19866328, 0.01483496, 0.0618...","[450, 4.5, 0]"
2,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ABL2,MVLGTVLLPPNSYGRDQDTSLCCLCTEASESALPDLTDHFASCVED...,10000.0,-4.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.028185444, -0.17497379, 0.061383072, 0.063...","[450, 4.5, 0]"
3,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ACVR1,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,10000.0,-4.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[0.060061518, -0.12135677, 0.12382316, 0.06401...","[450, 4.5, 0]"
4,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ACVR2A,MGAAAKLAFAVFLISCSSGAILGRSETQECLFFNANWEKDRTNQTG...,10000.0,-4.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[0.10380349, -0.11144593, 0.11350971, 0.110970...","[450, 4.5, 0]"


In [ ]:
# Apply the extract_target_features function to the 'Target' column of davis_db
davis_db['Extracted_Target_Features'] = davis_db['Target'].apply(extract_target_features)

# Display the updated DataFrame with the new column
display(davis_db.head())

NameError: name 'extract_target_features' is not defined

In [ ]:
def extract_target_features(sequence):
    """Extracts features from a protein sequence."""
    if sequence is None:
        return None
    # Example feature extraction: Sequence length and count of Alanine (A)
    seq_length = len(sequence)
    alanine_count = sequence.count('A')
    return [seq_length, alanine_count]

# Apply the extract_target_features function to the 'Target' column of davis_db
davis_db['Extracted_Target_Features'] = davis_db['Target'].apply(extract_target_features)

# Display the updated DataFrame with the new column
display(davis_db.head())

,Drug_ID,Drug,Target_ID,Target,Y,Affinity,Drug_Features,Target_Features,Extracted_Drug_Features,Extracted_Target_Features
0,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,AAK1,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,43.0,-1.633468,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.009276379, -0.14671455, 0.065141745, 0.037...","[450, 4.5, 0]","[961, 79]"
1,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ABL1p,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,10000.0,-4.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.052989967, -0.19866328, 0.01483496, 0.0618...","[450, 4.5, 0]","[1167, 99]"
2,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ABL2,MVLGTVLLPPNSYGRDQDTSLCCLCTEASESALPDLTDHFASCVED...,10000.0,-4.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.028185444, -0.17497379, 0.061383072, 0.063...","[450, 4.5, 0]","[1167, 96]"
3,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ACVR1,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,10000.0,-4.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[0.060061518, -0.12135677, 0.12382316, 0.06401...","[450, 4.5, 0]","[509, 22]"
4,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ACVR2A,MGAAAKLAFAVFLISCSSGAILGRSETQECLFFNANWEKDRTNQTG...,10000.0,-4.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[0.10380349, -0.11144593, 0.11350971, 0.110970...","[450, 4.5, 0]","[513, 38]"
